## Step 1. Import modules

In [1]:
# import libraries and modules
from IPython.display import HTML
get_ipython().run_line_magic('matplotlib', 'notebook')
import matplotlib.pyplot as plt
from tabulate import tabulate
import pandas as pd
import numpy as np
import sys
import IPython.display as display
import warnings
warnings.filterwarnings('ignore')
import re
import matplotlib as mpl
from scipy import stats

from pandas.io.html import read_html

import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.optimize import curve_fit

## Step 2. Define functions

In [2]:
def logistic(x, a, b, c, d):
    return a / (d + np.exp(-c * x + b))

def obs_pred_rsquare(obs, pred):
    return 1 - sum((obs - pred) ** 2) / sum((obs - np.mean(obs)) ** 2)

def exponential(x, a, b, c, d):
    return a * np.exp(b * x) + c

## Step 3. Pull online data:

### 1. COVID-19: John's Hopkins (https://github.com/CSSEGISandData/COVID-19)
### 2. COVID-19: Sito del Dipartimento della Protezione Civile, Italia (https://github.com/pcm-dpc/COVID-19)
### 3. Population size: klocey/COVID-dash/master/worldcities.csv


In [3]:
JH_DATA_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-'
Italia_DATA_URL = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province.csv'
CITIES_URL = 'https://raw.githubusercontent.com/klocey/COVID-dash/master/worldcities.csv'


## Step 4. Process data into primary dataframes.

In [4]:
df_confirmed = pd.read_csv(JH_DATA_URL + 'Confirmed.csv', index_col=False)
ser = pd.Series(df_confirmed.index.tolist()).astype(str)
df_confirmed['Province/State'].fillna(ser + df_confirmed['Country/Region'],inplace=True)
df_confirmed['type'] = 'Confirmed'

df_deaths = pd.read_csv(JH_DATA_URL + 'Deaths.csv', index_col=False)
ser = pd.Series(df_deaths.index.tolist()).astype(str)
df_deaths['Province/State'].fillna(ser + df_deaths['Country/Region'],inplace=True)
df_deaths['type'] = 'Deaths'
        
df_recovered = pd.read_csv(JH_DATA_URL + 'Recovered.csv', index_col=False)
ser = pd.Series(df_recovered.index.tolist()).astype(str)
df_recovered['Province/State'].fillna(ser + df_recovered['Country/Region'],inplace=True)
df_recovered['type'] = 'Recovered'        
        
ProvState = df_confirmed['Province/State']
CountryRegion = df_confirmed['Country/Region']

In [5]:
WorldCities = pd.read_csv(CITIES_URL, index_col=False)
places = ProvState.tolist()

print(len(places))
print(df_confirmed.shape)
print(df_deaths.shape)
print(df_recovered.shape)

404
(404, 55)
(404, 55)
(404, 55)


In [6]:

popsizes = []
for place in places:
    popsize = 0
    p_ls = ['Taiwan', 'San Diego County', 'Humboldt County', 'Sacramento County']
    s_ls = [23545963, 3095313, 132646, 1418788]
        
    if place in p_ls:
        i = p_ls.index(place)
        popsize = s_ls[i]
        
    else:
        try:
            pattern = re.compile(r'\w\,')
            if pattern.findall(place):
                place = ','.join(place.split(',')[:-1])    
    
            subdf = WorldCities[WorldCities.isin([place]).any(1)]
            popsize = sum(subdf['population'])
        except:
            pass
    popsizes.append(popsize)
    
df_confirmed['population size'] = list(popsizes)
df_recovered['population size'] = list(popsizes)
df_deaths['population size'] = list(popsizes)

In [7]:
df = pd.concat([df_confirmed, df_deaths, df_recovered])

#df['city'] = [0]*df.shape[0]
#df['county'] = [0]*df.shape[0]
col1 = df.pop('type')
col2 = df.pop('population size')
#col3 = df.pop('county')
df.insert(0, 'type', col1)
df.insert(1, 'population size', col2)
#df.insert(2, 'county', col3)
print(list(df))

['type', 'population size', 'Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20']


In [8]:
print(df.shape)

(1212, 56)


In [9]:
yi = list(df)
dates = yi[6:]

In [10]:
print(list(dates))

['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20']


In [11]:
df_Italia = pd.read_csv(Italia_DATA_URL, sep='\,', error_bad_lines=False)
#print(list(df_confirmed_Italia))
#print(df_confirmed_Italia.shape)
#print(list(df))

In [12]:
#df_Milan = df_confirmed_Italia[df_confirmed_Italia['denominazione_provincia'] == 'Milano']

#print(list(df_Milan))
#print(df_Milan.iloc[0])
dates = df_Italia['data'].tolist()
#print(dates)

try:
    df_Italia['data'] = df_Italia['data'].map(lambda x: x.rstrip(u"18:0"))
except:
    pass
try:
    df_Italia['data'] = df_Italia['data'].map(lambda x: x.rstrip(u"17:0"))
except:
    pass

df_Italia['data'] = pd.to_datetime(df_Italia['data'])
df_Italia['data'] = df_Italia['data'].dt.strftime('%m/%d/%Y')
#df_Milan['data'] = df_Milan['data'].dt.date

try:
    df_Italia['data'] = df_Italia['data'].map(lambda x: x.rstrip('0'))
    df_Italia['data'] = df_Italia['data'].map(lambda x: x.rstrip('2'))
    df_Italia['data'] = df_Italia['data'].map(lambda x: x.lstrip('0'))
except:
    pass

#print(df_Italia['data'])
#print(df_Italia['totale_casi'])

In [13]:
# Merge Italia and JH dataframes

df_ItaliaT = df_Italia.T
#print(list(df_MilanT))
#df_Milan.insert(0, 'type', col1)
#df_Milan.insert(1, 'population size', col2)

tab = tabulate(df_Italia.head(n=10), tablefmt="pipe", headers="keys")
display.display(display.Markdown(tab))


|    | data    | stato   |   codice_regione | denominazione_regione   |   codice_provincia | denominazione_provincia              | sigla_provincia   |     lat |    long |   totale_casi |
|---:|:--------|:--------|-----------------:|:------------------------|-------------------:|:-------------------------------------|:------------------|--------:|--------:|--------------:|
|  0 | 2/24/20 | ITA     |               13 | Abruzzo                 |                 69 | Chieti                               | CH                | 42.351  | 14.1675 |             0 |
|  1 | 2/24/20 | ITA     |               13 | Abruzzo                 |                 66 | L'Aquila                             | AQ                | 42.3512 | 13.3984 |             0 |
|  2 | 2/24/20 | ITA     |               13 | Abruzzo                 |                 68 | Pescara                              | PE                | 42.4646 | 14.2136 |             0 |
|  3 | 2/24/20 | ITA     |               13 | Abruzzo                 |                 67 | Teramo                               | TE                | 42.6589 | 13.7044 |             0 |
|  4 | 2/24/20 | ITA     |               13 | Abruzzo                 |                979 | In fase di definizione/aggiornamento | nan               |  0      |  0      |             0 |
|  5 | 2/24/20 | ITA     |               17 | Basilicata              |                 77 | Matera                               | MT                | 40.6675 | 16.5979 |             0 |
|  6 | 2/24/20 | ITA     |               17 | Basilicata              |                 76 | Potenza                              | PZ                | 40.6395 | 15.8051 |             0 |
|  7 | 2/24/20 | ITA     |               17 | Basilicata              |                980 | In fase di definizione/aggiornamento | nan               |  0      |  0      |             0 |
|  8 | 2/24/20 | ITA     |                4 | P.A. Bolzano            |                 21 | Bolzano                              | BZ                | 46.4993 | 11.3566 |             0 |
|  9 | 2/24/20 | ITA     |                4 | P.A. Bolzano            |                981 | In fase di definizione/aggiornamento | nan               |  0      |  0      |             0 |

In [14]:
Italia_df  = pd.DataFrame(columns = list(df))
print(list(Italia_df))

Italia_df['type'] = 'Confirmed'
Italia_df['population size'] = 0

['type', 'population size', 'Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20']


In [15]:
print(df_Italia.shape)

(2176, 10)


In [16]:
df_temp = df_Italia.filter(items=['denominazione_provincia', 'lat', 'long'])
df_temp.drop_duplicates(inplace=True)

Italia_df['Province/State'] = df_temp['denominazione_provincia']
Italia_df['Country/Region'] = 'Italia'
Italia_df['Lat'] = df_temp['lat']
Italia_df['Long'] = df_temp['long']
Italia_df = Italia_df.fillna(0)

tab = tabulate(Italia_df.head(n=10), tablefmt="pipe", headers="keys")
display.display(display.Markdown(tab))


|    |   type |   population size | Province/State                       | Country/Region   |     Lat |    Long |   1/22/20 |   1/23/20 |   1/24/20 |   1/25/20 |   1/26/20 |   1/27/20 |   1/28/20 |   1/29/20 |   1/30/20 |   1/31/20 |   2/1/20 |   2/2/20 |   2/3/20 |   2/4/20 |   2/5/20 |   2/6/20 |   2/7/20 |   2/8/20 |   2/9/20 |   2/10/20 |   2/11/20 |   2/12/20 |   2/13/20 |   2/14/20 |   2/15/20 |   2/16/20 |   2/17/20 |   2/18/20 |   2/19/20 |   2/20/20 |   2/21/20 |   2/22/20 |   2/23/20 |   2/24/20 |   2/25/20 |   2/26/20 |   2/27/20 |   2/28/20 |   2/29/20 |   3/1/20 |   3/2/20 |   3/3/20 |   3/4/20 |   3/5/20 |   3/6/20 |   3/7/20 |   3/8/20 |   3/9/20 |   3/10/20 |   3/11/20 |
|---:|-------:|------------------:|:-------------------------------------|:-----------------|--------:|--------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|
|  0 |      0 |                 0 | Chieti                               | Italia           | 42.351  | 14.1675 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  1 |      0 |                 0 | L'Aquila                             | Italia           | 42.3512 | 13.3984 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  2 |      0 |                 0 | Pescara                              | Italia           | 42.4646 | 14.2136 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  3 |      0 |                 0 | Teramo                               | Italia           | 42.6589 | 13.7044 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  4 |      0 |                 0 | In fase di definizione/aggiornamento | Italia           |  0      |  0      |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  5 |      0 |                 0 | Matera                               | Italia           | 40.6675 | 16.5979 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  6 |      0 |                 0 | Potenza                              | Italia           | 40.6395 | 15.8051 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  8 |      0 |                 0 | Bolzano                              | Italia           | 46.4993 | 11.3566 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
| 10 |      0 |                 0 | Catanzaro                            | Italia           | 38.906  | 16.5944 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
| 11 |      0 |                 0 | Cosenza                              | Italia           | 39.2931 | 16.2561 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |

In [17]:
df_Italia = df_Italia.fillna(0)
for index, row in Italia_df.iterrows():
    p = row['Province/State']
    df_temp = df_Italia[df_Italia['denominazione_provincia'] == p] 
    dates = df_temp['data'].tolist()
    
    cases = df_temp['totale_casi'].tolist()
    
    for i, date in enumerate(dates):
        Italia_df.set_value(index, date, int(cases[i]))

#print(np.max(Italia_df['3/11/20']))

In [18]:
tab = tabulate(Italia_df.head(n=10), tablefmt="pipe", headers="keys")
display.display(display.Markdown(tab))

|    |   type |   population size | Province/State                       | Country/Region   |     Lat |    Long |   1/22/20 |   1/23/20 |   1/24/20 |   1/25/20 |   1/26/20 |   1/27/20 |   1/28/20 |   1/29/20 |   1/30/20 |   1/31/20 |   2/1/20 |   2/2/20 |   2/3/20 |   2/4/20 |   2/5/20 |   2/6/20 |   2/7/20 |   2/8/20 |   2/9/20 |   2/10/20 |   2/11/20 |   2/12/20 |   2/13/20 |   2/14/20 |   2/15/20 |   2/16/20 |   2/17/20 |   2/18/20 |   2/19/20 |   2/20/20 |   2/21/20 |   2/22/20 |   2/23/20 |   2/24/20 |   2/25/20 |   2/26/20 |   2/27/20 |   2/28/20 |   2/29/20 |   3/1/20 |   3/2/20 |   3/3/20 |   3/4/20 |   3/5/20 |   3/6/20 |   3/7/20 |   3/8/20 |   3/9/20 |   3/10/20 |   3/11/20 |   3/01/20 |   3/02/20 |   3/03/20 |   3/04/20 |   3/05/20 |   3/06/20 |   3/07/20 |   3/08/20 |   3/09/20 |
|---:|-------:|------------------:|:-------------------------------------|:-----------------|--------:|--------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|
|  0 |      0 |                 0 | Chieti                               | Italia           | 42.351  | 14.1675 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         9 |         9 |         0 |         0 |         0 |         0 |         2 |         3 |         4 |         4 |         7 |
|  1 |      0 |                 0 | L'Aquila                             | Italia           | 42.3512 | 13.3984 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         6 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         1 |         5 |
|  2 |      0 |                 0 | Pescara                              | Italia           | 42.4646 | 14.2136 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        18 |        18 |         0 |         0 |         1 |         1 |         1 |         1 |         2 |         8 |        14 |
|  3 |      0 |                 0 | Teramo                               | Italia           | 42.6589 | 13.7044 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         2 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         5 |         5 |         5 |         5 |         3 |         3 |         4 |         4 |         4 |         4 |         4 |
|  4 |      0 |                 0 | In fase di definizione/aggiornamento | Italia           |  0      |  0      |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         5 |        21 |        13 |        42 |        53 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        42 |        40 |        86 |         4 |         5 |        11 |        14 |        24 |        12 |        22 |        43 |
|  5 |      0 |                 0 | Matera                               | Italia           | 40.6675 | 16.5979 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         3 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         2 |         3 |
|  6 |      0 |                 0 | Potenza                              | Italia           | 40.6395 | 15.8051 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         4 |         5 |         0 |         0 |         1 |         1 |         1 |         2 |         2 |         2 |         2 |
|  8 |      0 |                 0 | Bolzano                              | Italia           | 46.4993 | 11.3566 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        38 |        75 |         1 |         1 |         1 |         1 |         1 |         4 |         9 |         9 |         9 |
| 10 |      0 |                 0 | Catanzaro                            | Italia           | 38.906  | 16.5944 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         2 |         0 |         0 |         0 |         0 |         1 |         2 |         2 |         2 |         3 |
| 11 |      0 |                 0 | Cosenza                              | Italia           | 39.2931 | 16.2561 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         5 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         4 |

In [19]:
Italia_df = Italia_df[df.columns]

Italia_df['type'] = ['Confirmed']*Italia_df.shape[0]

tab = tabulate(Italia_df.head(n=10), tablefmt="pipe", headers="keys")
display.display(display.Markdown(tab))

#print(list(Italia_df))
#print(Italia_df['3/10/20'].tolist())

|    | type      |   population size | Province/State                       | Country/Region   |     Lat |    Long |   1/22/20 |   1/23/20 |   1/24/20 |   1/25/20 |   1/26/20 |   1/27/20 |   1/28/20 |   1/29/20 |   1/30/20 |   1/31/20 |   2/1/20 |   2/2/20 |   2/3/20 |   2/4/20 |   2/5/20 |   2/6/20 |   2/7/20 |   2/8/20 |   2/9/20 |   2/10/20 |   2/11/20 |   2/12/20 |   2/13/20 |   2/14/20 |   2/15/20 |   2/16/20 |   2/17/20 |   2/18/20 |   2/19/20 |   2/20/20 |   2/21/20 |   2/22/20 |   2/23/20 |   2/24/20 |   2/25/20 |   2/26/20 |   2/27/20 |   2/28/20 |   2/29/20 |   3/1/20 |   3/2/20 |   3/3/20 |   3/4/20 |   3/5/20 |   3/6/20 |   3/7/20 |   3/8/20 |   3/9/20 |   3/10/20 |   3/11/20 |
|---:|:----------|------------------:|:-------------------------------------|:-----------------|--------:|--------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|
|  0 | Confirmed |                 0 | Chieti                               | Italia           | 42.351  | 14.1675 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         9 |         9 |
|  1 | Confirmed |                 0 | L'Aquila                             | Italia           | 42.3512 | 13.3984 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         6 |
|  2 | Confirmed |                 0 | Pescara                              | Italia           | 42.4646 | 14.2136 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        18 |        18 |
|  3 | Confirmed |                 0 | Teramo                               | Italia           | 42.6589 | 13.7044 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         2 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         5 |         5 |
|  4 | Confirmed |                 0 | In fase di definizione/aggiornamento | Italia           |  0      |  0      |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         5 |        21 |        13 |        42 |        53 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        42 |        40 |
|  5 | Confirmed |                 0 | Matera                               | Italia           | 40.6675 | 16.5979 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         3 |
|  6 | Confirmed |                 0 | Potenza                              | Italia           | 40.6395 | 15.8051 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         4 |         5 |
|  8 | Confirmed |                 0 | Bolzano                              | Italia           | 46.4993 | 11.3566 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        38 |        75 |
| 10 | Confirmed |                 0 | Catanzaro                            | Italia           | 38.906  | 16.5944 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         2 |
| 11 | Confirmed |                 0 | Cosenza                              | Italia           | 39.2931 | 16.2561 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         5 |

In [20]:
tab = tabulate(df.head(n=10), tablefmt="pipe", headers="keys")
display.display(display.Markdown(tab))

|    | type      |   population size | Province/State   | Country/Region   |      Lat |     Long |   1/22/20 |   1/23/20 |   1/24/20 |   1/25/20 |   1/26/20 |   1/27/20 |   1/28/20 |   1/29/20 |   1/30/20 |   1/31/20 |   2/1/20 |   2/2/20 |   2/3/20 |   2/4/20 |   2/5/20 |   2/6/20 |   2/7/20 |   2/8/20 |   2/9/20 |   2/10/20 |   2/11/20 |   2/12/20 |   2/13/20 |   2/14/20 |   2/15/20 |   2/16/20 |   2/17/20 |   2/18/20 |   2/19/20 |   2/20/20 |   2/21/20 |   2/22/20 |   2/23/20 |   2/24/20 |   2/25/20 |   2/26/20 |   2/27/20 |   2/28/20 |   2/29/20 |   3/1/20 |   3/2/20 |   3/3/20 |   3/4/20 |   3/5/20 |   3/6/20 |   3/7/20 |   3/8/20 |   3/9/20 |   3/10/20 |   3/11/20 |
|---:|:----------|------------------:|:-----------------|:-----------------|---------:|---------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|
|  0 | Confirmed |       0           | 0Thailand        | Thailand         |  15      |  101     |         2 |         3 |         5 |         7 |         8 |         8 |        14 |        14 |        14 |        19 |       19 |       19 |       19 |       25 |       25 |       25 |       25 |       32 |       32 |        32 |        33 |        33 |        33 |        33 |        33 |        34 |        35 |        35 |        35 |        35 |        35 |        35 |        35 |        35 |        37 |        40 |        40 |        41 |        42 |       42 |       43 |       43 |       43 |       47 |       48 |       50 |       50 |       50 |        53 |        59 |
|  1 | Confirmed |       0           | 1Japan           | Japan            |  36      |  138     |         2 |         1 |         2 |         2 |         4 |         4 |         7 |         7 |        11 |        15 |       20 |       20 |       20 |       22 |       22 |       45 |       25 |       25 |       26 |        26 |        26 |        28 |        28 |        29 |        43 |        59 |        66 |        74 |        84 |        94 |       105 |       122 |       147 |       159 |       170 |       189 |       214 |       228 |       241 |      256 |      274 |      293 |      331 |      360 |      420 |      461 |      502 |      511 |       581 |       639 |
|  2 | Confirmed |       0           | 2Singapore       | Singapore        |   1.2833 |  103.833 |         0 |         1 |         3 |         3 |         4 |         5 |         7 |         7 |        10 |        13 |       16 |       18 |       18 |       24 |       28 |       28 |       30 |       33 |       40 |        45 |        47 |        50 |        58 |        67 |        72 |        75 |        77 |        81 |        84 |        84 |        85 |        85 |        89 |        89 |        91 |        93 |        93 |        93 |       102 |      106 |      108 |      110 |      110 |      117 |      130 |      138 |      150 |      150 |       160 |       178 |
|  3 | Confirmed |       0           | 3Nepal           | Nepal            |  28.1667 |   84.25  |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
|  4 | Confirmed |       0           | 4Malaysia        | Malaysia         |   2.5    |  112.5   |         0 |         0 |         0 |         3 |         4 |         4 |         4 |         7 |         8 |         8 |        8 |        8 |        8 |       10 |       12 |       12 |       12 |       16 |       16 |        18 |        18 |        18 |        19 |        19 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        23 |        23 |        25 |       29 |       29 |       36 |       50 |       50 |       83 |       93 |       99 |      117 |       129 |       149 |
|  5 | Confirmed |       3.42642e+06 | British Columbia | Canada           |  49.2827 | -123.121 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        2 |        2 |        4 |        4 |        4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         5 |         5 |         5 |         5 |         6 |         6 |         6 |         6 |         7 |         7 |         7 |         7 |         8 |        8 |        8 |        9 |       12 |       13 |       21 |       21 |       27 |       32 |        32 |        39 |
|  6 | Confirmed |       7.02054e+06 | New South Wales  | Australia        | -33.8688 |  151.209 |         0 |         0 |         0 |         0 |         3 |         4 |         4 |         4 |         4 |         4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |        6 |        6 |       13 |       22 |       22 |       26 |       28 |       38 |       48 |        55 |        65 |
|  7 | Confirmed |     nan           | Victoria         | Australia        | -37.8136 |  144.963 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         2 |         3 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         7 |        7 |        9 |        9 |       10 |       10 |       10 |       11 |       11 |       15 |        18 |        21 |
|  8 | Confirmed |       3.32551e+06 | Queensland       | Australia        | -28.0167 |  153.4   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         2 |        3 |        2 |        2 |        3 |        3 |        4 |        5 |        5 |        5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         9 |        9 |        9 |       11 |       11 |       13 |       13 |       13 |       15 |       15 |        18 |        20 |
|  9 | Confirmed |       0           | 9Cambodia        | Cambodia         |  11.55   |  104.917 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        2 |        2 |         2 |         3 |

In [21]:
print(df.shape)
print(Italia_df.shape)
#print(df_Italia.shape)

(1212, 56)
(108, 56)


In [22]:
dfMain = pd.concat([Italia_df, df], ignore_index=True)

In [23]:
tab = tabulate(dfMain.head(n=200), tablefmt="pipe", headers="keys")
display.display(display.Markdown(tab))

|     | type      |   population size | Province/State                       | Country/Region         |      Lat |       Long |   1/22/20 |   1/23/20 |   1/24/20 |   1/25/20 |   1/26/20 |   1/27/20 |   1/28/20 |   1/29/20 |   1/30/20 |   1/31/20 |   2/1/20 |   2/2/20 |   2/3/20 |   2/4/20 |   2/5/20 |   2/6/20 |   2/7/20 |   2/8/20 |   2/9/20 |   2/10/20 |   2/11/20 |   2/12/20 |   2/13/20 |   2/14/20 |   2/15/20 |   2/16/20 |   2/17/20 |   2/18/20 |   2/19/20 |   2/20/20 |   2/21/20 |   2/22/20 |   2/23/20 |   2/24/20 |   2/25/20 |   2/26/20 |   2/27/20 |   2/28/20 |   2/29/20 |   3/1/20 |   3/2/20 |   3/3/20 |   3/4/20 |   3/5/20 |   3/6/20 |   3/7/20 |   3/8/20 |   3/9/20 |   3/10/20 |   3/11/20 |
|----:|:----------|------------------:|:-------------------------------------|:-----------------------|---------:|-----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|
|   0 | Confirmed |       0           | Chieti                               | Italia                 |  42.351  |   14.1675  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         9 |         9 |
|   1 | Confirmed |       0           | L'Aquila                             | Italia                 |  42.3512 |   13.3984  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         6 |
|   2 | Confirmed |       0           | Pescara                              | Italia                 |  42.4646 |   14.2136  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        18 |        18 |
|   3 | Confirmed |       0           | Teramo                               | Italia                 |  42.6589 |   13.7044  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         2 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         5 |         5 |
|   4 | Confirmed |       0           | In fase di definizione/aggiornamento | Italia                 |   0      |    0       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         5 |        21 |        13 |        42 |        53 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        42 |        40 |
|   5 | Confirmed |       0           | Matera                               | Italia                 |  40.6675 |   16.5979  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         3 |
|   6 | Confirmed |       0           | Potenza                              | Italia                 |  40.6395 |   15.8051  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         4 |         5 |
|   7 | Confirmed |       0           | Bolzano                              | Italia                 |  46.4993 |   11.3566  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        38 |        75 |
|   8 | Confirmed |       0           | Catanzaro                            | Italia                 |  38.906  |   16.5944  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         2 |
|   9 | Confirmed |       0           | Cosenza                              | Italia                 |  39.2931 |   16.2561  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         5 |
|  10 | Confirmed |       0           | Crotone                              | Italia                 |  39.0804 |   17.1254  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         1 |
|  11 | Confirmed |       0           | Reggio di Calabria                   | Italia                 |  38.1092 |   15.6435  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         6 |
|  12 | Confirmed |       0           | Vibo Valentia                        | Italia                 |  38.6762 |   16.1016  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         5 |
|  13 | Confirmed |       0           | Avellino                             | Italia                 |  40.914  |   14.7953  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         7 |
|  14 | Confirmed |       0           | Benevento                            | Italia                 |  41.1297 |   14.7815  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         4 |         2 |
|  15 | Confirmed |       0           | Caserta                              | Italia                 |  41.0747 |   14.3324  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        34 |        26 |
|  16 | Confirmed |       0           | Napoli                               | Italia                 |  40.8396 |   14.2508  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         4 |        13 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        61 |        96 |
|  17 | Confirmed |       0           | Salerno                              | Italia                 |  40.6782 |   14.7594  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        17 |        17 |
|  18 | Confirmed |       0           | Bologna                              | Italia                 |  44.4944 |   11.3417  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        86 |       108 |
|  19 | Confirmed |       0           | Ferrara                              | Italia                 |  44.836  |   11.6187  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         8 |        12 |
|  20 | Confirmed |       0           | Forlì-Cesena                         | Italia                 |  44.2227 |   12.0407  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        20 |        24 |
|  21 | Confirmed |       0           | Modena                               | Italia                 |  44.646  |   10.9262  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         8 |        18 |        18 |        22 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       127 |       163 |
|  22 | Confirmed |       0           | Parma                                | Italia                 |  44.8011 |   10.3283  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         4 |         8 |        10 |        27 |        35 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       325 |       378 |
|  23 | Confirmed |       0           | Piacenza                             | Italia                 |  45.0519 |    9.69263 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        18 |        28 |        63 |        89 |       138 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       633 |       664 |
|  24 | Confirmed |       0           | Ravenna                              | Italia                 |  44.4172 |   12.1991  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        24 |        31 |
|  25 | Confirmed |       0           | Reggio nell'Emilia                   | Italia                 |  44.6974 |   10.6301  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         4 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       104 |       114 |
|  26 | Confirmed |       0           | Rimini                               | Italia                 |  44.0609 |   12.5656  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         6 |         9 |        15 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       206 |       245 |
|  27 | Confirmed |       0           | Gorizia                              | Italia                 |  45.9415 |   13.6221  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        10 |        12 |
|  28 | Confirmed |       0           | Pordenone                            | Italia                 |  45.9544 |   12.66    |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         4 |        13 |
|  29 | Confirmed |       0           | Trieste                              | Italia                 |  45.6494 |   13.7681  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        53 |        57 |
|  30 | Confirmed |       0           | Udine                                | Italia                 |  46.0626 |   13.2348  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        26 |        44 |
|  31 | Confirmed |       0           | Frosinone                            | Italia                 |  41.6396 |   13.3512  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         8 |
|  32 | Confirmed |       0           | Latina                               | Italia                 |  41.4676 |   12.9037  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        11 |        11 |
|  33 | Confirmed |       0           | Rieti                                | Italia                 |  42.4049 |   12.8621  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         3 |
|  34 | Confirmed |       0           | Roma                                 | Italia                 |  41.8928 |   12.4837  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         3 |         3 |         3 |         6 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        76 |        99 |
|  35 | Confirmed |       0           | Viterbo                              | Italia                 |  42.4174 |   12.1047  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         5 |        10 |
|  36 | Confirmed |       0           | Genova                               | Italia                 |  44.4115 |    8.9327  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        42 |        63 |
|  37 | Confirmed |       0           | Imperia                              | Italia                 |  43.8857 |    8.02785 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        15 |        18 |
|  38 | Confirmed |       0           | La Spezia                            | Italia                 |  44.107  |    9.82819 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        15 |        19 |
|  39 | Confirmed |       0           | Savona                               | Italia                 |  44.3075 |    8.48111 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        10 |        18 |        18 |        41 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        38 |        42 |
|  40 | Confirmed |       0           | Bergamo                              | Italia                 |  45.6944 |    9.66842 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        18 |        20 |        72 |       103 |       110 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |      1472 |      1815 |
|  41 | Confirmed |       0           | Brescia                              | Italia                 |  45.5399 |   10.2191  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |        10 |        13 |        14 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       790 |      1351 |
|  42 | Confirmed |       0           | Como                                 | Italia                 |  45.81   |    9.08516 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        46 |        77 |
|  43 | Confirmed |       0           | Cremona                              | Italia                 |  45.1334 |   10.0242  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        53 |        57 |        91 |       123 |       136 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       957 |      1061 |
|  44 | Confirmed |       0           | Lecco                                | Italia                 |  45.8558 |    9.39339 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        89 |       113 |
|  45 | Confirmed |       0           | Lodi                                 | Italia                 |  45.3144 |    9.50372 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |       125 |       128 |       159 |       182 |       237 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       963 |      1035 |
|  46 | Confirmed |       0           | Mantova                              | Italia                 |  45.1573 |   10.7928  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       119 |       137 |
|  47 | Confirmed |       0           | Milano                               | Italia                 |  45.4668 |    9.19035 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         8 |         8 |        15 |        29 |        30 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       592 |       925 |
|  48 | Confirmed |       0           | Monza e della Brianza                | Italia                 |  45.5844 |    9.27358 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         4 |         5 |         6 |         6 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        65 |        85 |
|  49 | Confirmed |       0           | Pavia                                | Italia                 |  45.1851 |    9.16016 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        27 |        27 |        36 |        49 |        55 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       324 |       403 |
|  50 | Confirmed |       0           | Sondrio                              | Italia                 |  46.171  |    9.87147 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         3 |         3 |         3 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         7 |        13 |
|  51 | Confirmed |       0           | Varese                               | Italia                 |  45.817  |    8.82287 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         4 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        50 |        75 |
|  52 | Confirmed |       0           | Ancona                               | Italia                 |  43.6168 |   13.5189  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        81 |       110 |
|  53 | Confirmed |       0           | Ascoli Piceno                        | Italia                 |  42.8532 |   13.5769  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         1 |
|  54 | Confirmed |       0           | Fermo                                | Italia                 |  43.1606 |   13.7184  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         8 |
|  55 | Confirmed |       0           | Macerata                             | Italia                 |  43.3002 |   13.4531  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        11 |        17 |
|  56 | Confirmed |       0           | Pesaro e Urbino                      | Italia                 |  43.9101 |   12.9135  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         6 |        11 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       296 |       342 |
|  57 | Confirmed |       0           | Campobasso                           | Italia                 |  41.5577 |   14.6592  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        15 |        16 |
|  58 | Confirmed |       0           | Isernia                              | Italia                 |  41.588  |   14.2258  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  59 | Confirmed |       0           | Alessandria                          | Italia                 |  44.913  |    8.6154  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        65 |       105 |
|  60 | Confirmed |       0           | Asti                                 | Italia                 |  44.8991 |    8.20414 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        58 |        68 |
|  61 | Confirmed |       0           | Biella                               | Italia                 |  45.5665 |    8.05408 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        20 |        36 |
|  62 | Confirmed |       0           | Cuneo                                | Italia                 |  44.3933 |    7.55117 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        14 |        17 |
|  63 | Confirmed |       0           | Novara                               | Italia                 |  45.4459 |    8.62192 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        22 |        27 |
|  64 | Confirmed |       0           | Torino                               | Italia                 |  45.0733 |    7.68069 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         3 |         2 |        11 |        11 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       111 |       159 |
|  65 | Confirmed |       0           | Verbano-Cusio-Ossola                 | Italia                 |  45.9214 |    8.55108 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        11 |        13 |
|  66 | Confirmed |       0           | Vercelli                             | Italia                 |  45.324  |    8.42323 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        18 |        24 |
|  67 | Confirmed |       0           | Bari                                 | Italia                 |  41.1256 |   16.8674  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        15 |        18 |
|  68 | Confirmed |       0           | Barletta-Andria-Trani                | Italia                 |  41.2271 |   16.2952  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         4 |
|  69 | Confirmed |       0           | Brindisi                             | Italia                 |  40.6385 |   17.946   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         5 |        15 |
|  70 | Confirmed |       0           | Foggia                               | Italia                 |  41.4623 |   15.5431  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        23 |        24 |
|  71 | Confirmed |       0           | Lecce                                | Italia                 |  40.3535 |   18.1719  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        10 |        12 |
|  72 | Confirmed |       0           | Taranto                              | Italia                 |  40.4735 |   17.2324  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         3 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         4 |
|  73 | Confirmed |       0           | Cagliari                             | Italia                 |  39.2153 |    9.11062 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        15 |        15 |
|  74 | Confirmed |       0           | Nuoro                                | Italia                 |  40.3232 |    9.3303  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |        18 |
|  75 | Confirmed |       0           | Oristano                             | Italia                 |  39.9038 |    8.59118 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         2 |
|  76 | Confirmed |       0           | Sassari                              | Italia                 |  40.7267 |    8.55967 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         2 |
|  77 | Confirmed |       0           | Sud Sardegna                         | Italia                 |  39.1664 |    8.52624 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  78 | Confirmed |       0           | Agrigento                            | Italia                 |  37.3097 |   13.5846  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        11 |        16 |
|  79 | Confirmed |       0           | Caltanissetta                        | Italia                 |  37.4921 |   14.0618  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  80 | Confirmed |       0           | Catania                              | Italia                 |  37.5029 |   15.087   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        35 |        41 |
|  81 | Confirmed |       0           | Enna                                 | Italia                 |  37.5671 |   14.2791  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         1 |
|  82 | Confirmed |       0           | Messina                              | Italia                 |  38.194  |   15.5557  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         2 |         4 |
|  83 | Confirmed |       0           | Palermo                              | Italia                 |  38.1157 |   13.3624  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         3 |         3 |         3 |         3 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        10 |        15 |
|  84 | Confirmed |       0           | Ragusa                               | Italia                 |  36.9251 |   14.7307  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         1 |
|  85 | Confirmed |       0           | Siracusa                             | Italia                 |  37.0599 |   15.2933  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         2 |         3 |
|  86 | Confirmed |       0           | Trapani                              | Italia                 |  38.0185 |   12.5137  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         2 |
|  87 | Confirmed |       0           | Arezzo                               | Italia                 |  43.4664 |   11.8823  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        13 |        14 |
|  88 | Confirmed |       0           | Firenze                              | Italia                 |  43.7692 |   11.2559  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         2 |         1 |         7 |         5 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        61 |        71 |
|  89 | Confirmed |       0           | Grosseto                             | Italia                 |  42.7603 |   11.1136  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        10 |        12 |
|  90 | Confirmed |       0           | Livorno                              | Italia                 |  43.5523 |   10.3087  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        14 |        16 |
|  91 | Confirmed |       0           | Lucca                                | Italia                 |  43.8443 |   10.5015  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        37 |        43 |
|  92 | Confirmed |       0           | Massa Carrara                        | Italia                 |  44.0367 |   10.1417  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        37 |        40 |
|  93 | Confirmed |       0           | Pisa                                 | Italia                 |  43.7155 |   10.4013  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        31 |        34 |
|  94 | Confirmed |       0           | Pistoia                              | Italia                 |  43.9335 |   10.9173  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         0 |         1 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        21 |        32 |
|  95 | Confirmed |       0           | Prato                                | Italia                 |  43.8806 |   11.097   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         7 |        21 |
|  96 | Confirmed |       0           | Siena                                | Italia                 |  43.3182 |   11.3319  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        33 |        37 |
|  97 | Confirmed |       0           | Trento                               | Italia                 |  46.0689 |   11.1212  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        52 |        77 |
|  98 | Confirmed |       0           | Perugia                              | Italia                 |  43.1068 |   12.3882  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        13 |        26 |
|  99 | Confirmed |       0           | Terni                                | Italia                 |  42.5607 |   12.6467  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        24 |        18 |
| 100 | Confirmed |       0           | Aosta                                | Italia                 |  45.7375 |    7.32015 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        17 |        20 |
| 101 | Confirmed |       0           | Belluno                              | Italia                 |  46.1384 |   12.217   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        29 |        30 |
| 102 | Confirmed |       0           | Padova                               | Italia                 |  45.4069 |   11.8761  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        30 |        40 |        59 |        68 |        81 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       296 |       373 |
| 103 | Confirmed |       0           | Rovigo                               | Italia                 |  45.0711 |   11.7901  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        10 |        14 |
| 104 | Confirmed |       0           | Treviso                              | Italia                 |  45.6675 |   12.2451  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         2 |        22 |        23 |        35 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       158 |       185 |
| 105 | Confirmed |       0           | Venezia                              | Italia                 |  45.4349 |   12.3385  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         7 |         8 |        14 |        15 |        19 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       152 |       179 |
| 106 | Confirmed |       0           | Verona                               | Italia                 |  45.4384 |   10.9935  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        96 |       110 |
| 107 | Confirmed |       0           | Vicenza                              | Italia                 |  45.5475 |   11.546   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         3 |         3 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        73 |        92 |
| 108 | Confirmed |       0           | 0Thailand                            | Thailand               |  15      |  101       |         2 |         3 |         5 |         7 |         8 |         8 |        14 |        14 |        14 |        19 |       19 |       19 |       19 |       25 |       25 |       25 |       25 |       32 |       32 |        32 |        33 |        33 |        33 |        33 |        33 |        34 |        35 |        35 |        35 |        35 |        35 |        35 |        35 |        35 |        37 |        40 |        40 |        41 |        42 |       42 |       43 |       43 |       43 |       47 |       48 |       50 |       50 |       50 |        53 |        59 |
| 109 | Confirmed |       0           | 1Japan                               | Japan                  |  36      |  138       |         2 |         1 |         2 |         2 |         4 |         4 |         7 |         7 |        11 |        15 |       20 |       20 |       20 |       22 |       22 |       45 |       25 |       25 |       26 |        26 |        26 |        28 |        28 |        29 |        43 |        59 |        66 |        74 |        84 |        94 |       105 |       122 |       147 |       159 |       170 |       189 |       214 |       228 |       241 |      256 |      274 |      293 |      331 |      360 |      420 |      461 |      502 |      511 |       581 |       639 |
| 110 | Confirmed |       0           | 2Singapore                           | Singapore              |   1.2833 |  103.833   |         0 |         1 |         3 |         3 |         4 |         5 |         7 |         7 |        10 |        13 |       16 |       18 |       18 |       24 |       28 |       28 |       30 |       33 |       40 |        45 |        47 |        50 |        58 |        67 |        72 |        75 |        77 |        81 |        84 |        84 |        85 |        85 |        89 |        89 |        91 |        93 |        93 |        93 |       102 |      106 |      108 |      110 |      110 |      117 |      130 |      138 |      150 |      150 |       160 |       178 |
| 111 | Confirmed |       0           | 3Nepal                               | Nepal                  |  28.1667 |   84.25    |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 112 | Confirmed |       0           | 4Malaysia                            | Malaysia               |   2.5    |  112.5     |         0 |         0 |         0 |         3 |         4 |         4 |         4 |         7 |         8 |         8 |        8 |        8 |        8 |       10 |       12 |       12 |       12 |       16 |       16 |        18 |        18 |        18 |        19 |        19 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        23 |        23 |        25 |       29 |       29 |       36 |       50 |       50 |       83 |       93 |       99 |      117 |       129 |       149 |
| 113 | Confirmed |       3.42642e+06 | British Columbia                     | Canada                 |  49.2827 | -123.121   |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        2 |        2 |        4 |        4 |        4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         5 |         5 |         5 |         5 |         6 |         6 |         6 |         6 |         7 |         7 |         7 |         7 |         8 |        8 |        8 |        9 |       12 |       13 |       21 |       21 |       27 |       32 |        32 |        39 |
| 114 | Confirmed |       7.02054e+06 | New South Wales                      | Australia              | -33.8688 |  151.209   |         0 |         0 |         0 |         0 |         3 |         4 |         4 |         4 |         4 |         4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |        6 |        6 |       13 |       22 |       22 |       26 |       28 |       38 |       48 |        55 |        65 |
| 115 | Confirmed |     nan           | Victoria                             | Australia              | -37.8136 |  144.963   |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         2 |         3 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         7 |        7 |        9 |        9 |       10 |       10 |       10 |       11 |       11 |       15 |        18 |        21 |
| 116 | Confirmed |       3.32551e+06 | Queensland                           | Australia              | -28.0167 |  153.4     |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         2 |        3 |        2 |        2 |        3 |        3 |        4 |        5 |        5 |        5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         9 |        9 |        9 |       11 |       11 |       13 |       13 |       13 |       15 |       15 |        18 |        20 |
| 117 | Confirmed |       0           | 9Cambodia                            | Cambodia               |  11.55   |  104.917   |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        2 |        2 |         2 |         3 |
| 118 | Confirmed |       0           | 10Sri Lanka                          | Sri Lanka              |   7      |   81       |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         2 |
| 119 | Confirmed |       0           | 11Germany                            | Germany                |  51      |    9       |         0 |         0 |         0 |         0 |         0 |         1 |         4 |         4 |         4 |         5 |        8 |       10 |       12 |       12 |       12 |       12 |       13 |       13 |       14 |        14 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        17 |        27 |        46 |        48 |        79 |      130 |      159 |      196 |      262 |      482 |      670 |      799 |     1040 |     1176 |      1457 |      1908 |
| 120 | Confirmed |       0           | 12Finland                            | Finland                |  64      |   26       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         2 |         2 |         2 |         3 |        6 |        6 |        6 |        6 |       12 |       15 |       15 |       23 |       30 |        40 |        59 |
| 121 | Confirmed |       0           | 13United Arab Emirates               | United Arab Emirates   |  24      |   54       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         4 |         4 |         4 |        4 |        5 |        5 |        5 |        5 |        5 |        5 |        7 |        7 |         8 |         8 |         8 |         8 |         8 |         8 |         9 |         9 |         9 |         9 |         9 |         9 |        13 |        13 |        13 |        13 |        13 |        13 |        19 |        21 |       21 |       21 |       27 |       27 |       29 |       29 |       45 |       45 |       45 |        74 |        74 |
| 122 | Confirmed |       0           | 14Philippines                        | Philippines            |  13      |  122       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        2 |        2 |        2 |        2 |        2 |        3 |        3 |        3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |        3 |        3 |        3 |        3 |        3 |        5 |        6 |       10 |       20 |        33 |        49 |
| 123 | Confirmed |       0           | 15India                              | India                  |  21      |   78       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        2 |        3 |        3 |        3 |        3 |        3 |        3 |        3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |        3 |        5 |        5 |       28 |       30 |       31 |       34 |       39 |       43 |        56 |        62 |
| 124 | Confirmed |       0           | 16Italy                              | Italy                  |  43      |   12       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |        2 |        2 |        2 |        2 |        2 |        2 |        3 |        3 |        3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |        20 |        62 |       155 |       229 |       322 |       453 |       655 |       888 |      1128 |     1694 |     2036 |     2502 |     3089 |     3858 |     4636 |     5883 |     7375 |     9172 |     10149 |     12462 |
| 125 | Confirmed |       0           | 17Sweden                             | Sweden                 |  63      |   16       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         2 |         7 |         7 |        12 |       14 |       15 |       21 |       35 |       94 |      101 |      161 |      203 |      248 |       355 |       500 |
| 126 | Confirmed |       0           | 18Spain                              | Spain                  |  40      |   -4       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         6 |        13 |        15 |        32 |        45 |       84 |      120 |      165 |      222 |      259 |      400 |      500 |      673 |     1073 |      1695 |      2277 |
| 127 | Confirmed |       1.2978e+06  | South Australia                      | Australia              | -34.9285 |  138.601   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        1 |        2 |        2 |        2 |        2 |        2 |        2 |        2 |        2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         3 |        3 |        3 |        3 |        5 |        5 |        7 |        7 |        7 |        7 |         7 |         9 |
| 128 | Confirmed |       0           | 20Belgium                            | Belgium                |  50.8333 |    4       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        2 |        8 |       13 |       23 |       50 |      109 |      169 |      200 |      239 |       267 |       314 |
| 129 | Confirmed |       0           | 21Egypt                              | Egypt                  |  26      |   30       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        2 |        2 |        2 |        2 |        3 |       15 |       15 |       49 |       55 |        59 |        60 |
| 130 | Confirmed |       0           | From Diamond Princess                | Australia              |  35.4437 |  139.638   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         4 |         7 |         7 |         7 |         7 |         7 |         8 |         8 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
| 131 | Confirmed |       0           | 23Lebanon                            | Lebanon                |  33.8547 |   35.8623  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         2 |         2 |         2 |         4 |       10 |       13 |       13 |       13 |       16 |       22 |       22 |       32 |       32 |        41 |        61 |
| 132 | Confirmed |       0           | 24Iraq                               | Iraq                   |  33      |   44       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         5 |         7 |         7 |        13 |       19 |       26 |       32 |       35 |       35 |       40 |       54 |       60 |       60 |        71 |        71 |
| 133 | Confirmed |       0           | 25Oman                               | Oman                   |  21      |   57       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |         2 |         4 |         4 |         4 |         6 |        6 |        6 |       12 |       15 |       16 |       16 |       16 |       16 |       16 |        18 |        18 |
| 134 | Confirmed |       0           | 26Afghanistan                        | Afghanistan            |  33      |   65       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        4 |        4 |         5 |         7 |
| 135 | Confirmed |       0           | 27Bahrain                            | Bahrain                |  26.0275 |   50.55    |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        23 |        33 |        33 |        36 |        41 |       47 |       49 |       49 |       52 |       55 |       60 |       85 |       85 |       95 |       110 |       195 |
| 136 | Confirmed |       0           | 28Kuwait                             | Kuwait                 |  29.5    |   47.75    |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        11 |        26 |        43 |        45 |        45 |       45 |       56 |       56 |       56 |       58 |       58 |       61 |       64 |       64 |        69 |        72 |
| 137 | Confirmed |       0           | 29Algeria                            | Algeria                |  28.0339 |    1.6596  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |        1 |        3 |        5 |       12 |       12 |       17 |       17 |       19 |       20 |        20 |        20 |
| 138 | Confirmed |       0           | 30Croatia                            | Croatia                |  45.1    |   15.2     |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         3 |         5 |         6 |        7 |        7 |        9 |       10 |       10 |       11 |       12 |       12 |       12 |        14 |        19 |
| 139 | Confirmed |       0           | 31Switzerland                        | Switzerland            |  46.8182 |    8.2275  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         8 |         8 |        18 |       27 |       42 |       56 |       90 |      114 |      214 |      268 |      337 |      374 |       491 |       652 |
| 140 | Confirmed |       0           | 32Austria                            | Austria                |  47.5162 |   14.5501  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |         2 |         3 |         3 |         9 |       14 |       18 |       21 |       29 |       41 |       55 |       79 |      104 |      131 |       182 |       246 |
| 141 | Confirmed |       0           | 33Israel                             | Israel                 |  31      |   35       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         2 |         3 |         4 |         7 |       10 |       10 |       12 |       15 |       20 |       37 |       43 |       61 |       61 |        83 |       109 |
| 142 | Confirmed |       0           | 34Pakistan                           | Pakistan               |  30.3753 |   69.3451  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |         2 |         2 |         4 |        4 |        4 |        5 |        5 |        5 |        6 |        6 |        6 |        6 |        16 |        19 |
| 143 | Confirmed |       0           | 35Brazil                             | Brazil                 | -14.235  |  -51.9253  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         2 |        2 |        2 |        2 |        4 |        4 |       13 |       13 |       20 |       25 |        31 |        38 |
| 144 | Confirmed |       0           | 36Georgia                            | Georgia                |  42.3154 |   43.3569  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |        3 |        3 |        3 |        3 |        4 |        4 |        4 |       13 |       15 |        15 |        24 |
| 145 | Confirmed |       0           | 37Greece                             | Greece                 |  39.0742 |   21.8243  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         4 |         4 |        7 |        7 |        7 |        9 |       31 |       45 |       46 |       73 |       73 |        89 |        99 |
| 146 | Confirmed |       0           | 38North Macedonia                    | North Macedonia        |  41.6086 |   21.7453  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        3 |        3 |        3 |        3 |         7 |         7 |
| 147 | Confirmed |       0           | 39Norway                             | Norway                 |  60.472  |    8.4689  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         6 |        15 |       19 |       25 |       32 |       56 |       87 |      108 |      147 |      176 |      205 |       400 |       598 |
| 148 | Confirmed |       0           | 40Romania                            | Romania                |  45.9432 |   24.9668  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         3 |         3 |        3 |        3 |        3 |        4 |        6 |        9 |        9 |       15 |       15 |        25 |        45 |
| 149 | Confirmed |       0           | 41Estonia                            | Estonia                |  58.5953 |   25.0136  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |        1 |        1 |        2 |        2 |        3 |       10 |       10 |       10 |       10 |        12 |        16 |
| 150 | Confirmed |       0           | 42Netherlands                        | Netherlands            |  52.1326 |    5.2913  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         6 |       10 |       18 |       24 |       38 |       82 |      128 |      188 |      265 |      321 |       382 |       503 |
| 151 | Confirmed |       0           | 43San Marino                         | San Marino             |  43.9424 |   12.4578  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |        1 |        8 |       10 |       16 |       21 |       21 |       23 |       36 |       36 |        51 |        62 |
| 152 | Confirmed |       0           | 44Belarus                            | Belarus                |  53.7098 |   27.9534  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        1 |        1 |        6 |        6 |        6 |        6 |        6 |        6 |         9 |         9 |
| 153 | Confirmed |       0           | 45Iceland                            | Iceland                |  64.9631 |  -19.0208  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        3 |        6 |       11 |       26 |       34 |       43 |       50 |       50 |       58 |        69 |        85 |
| 154 | Confirmed |       0           | 46Lithuania                          | Lithuania              |  55.1694 |   23.8813  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         3 |
| 155 | Confirmed |       0           | 47Mexico                             | Mexico                 |  23.6345 | -102.553   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         4 |        5 |        5 |        5 |        5 |        5 |        6 |        6 |        7 |        7 |         7 |         8 |
| 156 | Confirmed |       0           | 48New Zealand                        | New Zealand            | -40.9006 |  174.886   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        1 |        1 |        3 |        3 |        4 |        5 |        5 |        5 |         5 |         5 |
| 157 | Confirmed |       0           | 49Nigeria                            | Nigeria                |   9.082  |    8.6753  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        2 |         2 |         2 |
| 158 | Confirmed |       1.88282e+06 | Western Australia                    | Australia              | -31.9505 |  115.861   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |        2 |        2 |        2 |        2 |        3 |        3 |        3 |        3 |        4 |         6 |         9 |
| 159 | Confirmed |       0           | 51Ireland                            | Ireland                |  53.1424 |   -7.6921  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        1 |        1 |        2 |        6 |        6 |       18 |       18 |       19 |       21 |        34 |        43 |
| 160 | Confirmed |       0           | 52Luxembourg                         | Luxembourg             |  49.8153 |    6.1296  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        1 |        1 |        1 |        1 |        1 |        2 |        2 |        3 |        3 |         5 |         7 |
| 161 | Confirmed |       0           | 53Monaco                             | Monaco                 |  43.7333 |    7.4167  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 162 | Confirmed |       0           | 54Qatar                              | Qatar                  |  25.3548 |   51.1839  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        3 |        3 |        7 |        8 |        8 |        8 |        8 |       15 |       18 |        24 |       262 |
| 163 | Confirmed |       0           | 55Ecuador                            | Ecuador                |  -1.8312 |  -78.1834  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        6 |        6 |        7 |       10 |       13 |       13 |       13 |       14 |       15 |        15 |        17 |
| 164 | Confirmed |       0           | 56Azerbaijan                         | Azerbaijan             |  40.1431 |   47.5769  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        3 |        3 |        3 |        3 |        6 |        6 |        9 |        9 |        9 |        11 |        11 |
| 165 | Confirmed |       0           | 57Armenia                            | Armenia                |  40.0691 |   45.0382  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 166 | Confirmed |       0           | 58Dominican Republic                 | Dominican Republic     |  18.7357 |  -70.1627  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        1 |        1 |        1 |        1 |        1 |        2 |        2 |        5 |        5 |         5 |         5 |
| 167 | Confirmed |       0           | 59Indonesia                          | Indonesia              |  -0.7893 |  113.921   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        2 |        2 |        2 |        2 |        4 |        4 |        6 |       19 |        27 |        34 |
| 168 | Confirmed |       0           | 60Portugal                           | Portugal               |  39.3999 |   -8.2245  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        2 |        2 |        5 |        8 |       13 |       20 |       30 |       30 |        41 |        59 |
| 169 | Confirmed |       0           | 61Andorra                            | Andorra                |  42.5063 |    1.5218  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 170 | Confirmed |  217344           | Tasmania                             | Australia              | -41.4545 |  145.971   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |        2 |        2 |         2 |         3 |
| 171 | Confirmed |       0           | 63Latvia                             | Latvia                 |  56.8796 |   24.6032  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |        2 |        6 |         8 |        10 |
| 172 | Confirmed |       0           | 64Morocco                            | Morocco                |  31.7917 |   -7.0926  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        1 |        1 |        2 |        2 |        2 |        2 |        2 |         3 |         5 |
| 173 | Confirmed |       0           | 65Saudi Arabia                       | Saudi Arabia           |  24      |   45       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        1 |        1 |        5 |        5 |        5 |       11 |       15 |        20 |        21 |
| 174 | Confirmed |       0           | 66Senegal                            | Senegal                |  14.4974 |  -14.4524  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        2 |        4 |        4 |        4 |        4 |        4 |        4 |         4 |         4 |
| 175 | Confirmed |       0           | 67Argentina                          | Argentina              | -38.4161 |  -63.6167  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        1 |        1 |        1 |        2 |        8 |       12 |       12 |        17 |        19 |
| 176 | Confirmed |       0           | 68Chile                              | Chile                  | -35.6751 |  -71.543   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        1 |        1 |        4 |        4 |        4 |        8 |        8 |        13 |        23 |
| 177 | Confirmed |       0           | 69Jordan                             | Jordan                 |  31.24   |   36.51    |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 178 | Confirmed |       0           | 70Ukraine                            | Ukraine                |  48.3794 |   31.1656  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 179 | Confirmed |       0           | 71Hungary                            | Hungary                |  47.1625 |   19.5033  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        2 |        2 |        2 |        4 |        7 |        9 |         9 |        13 |
| 180 | Confirmed |  137788           | Northern Territory                   | Australia              | -12.4634 |  130.846   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        1 |        1 |        0 |        0 |        0 |        0 |         1 |         1 |
| 181 | Confirmed |       0           | 73Liechtenstein                      | Liechtenstein          |  47.14   |    9.55    |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 182 | Confirmed |       0           | 74Poland                             | Poland                 |  51.9194 |   19.1451  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        1 |        1 |        5 |        5 |       11 |       16 |        22 |        31 |
| 183 | Confirmed |       0           | 75Tunisia                            | Tunisia                |  34      |    9       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |        2 |        2 |         5 |         7 |
| 184 | Confirmed |       0           | 76Bosnia and Herzegovina             | Bosnia and Herzegovina |  43.9159 |   17.6791  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        2 |        2 |        3 |        3 |        3 |         5 |         7 |
| 185 | Confirmed |       0           | 77Slovenia                           | Slovenia               |  46.1512 |   14.9955  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        2 |        7 |        7 |       16 |       16 |        31 |        57 |
| 186 | Confirmed |       0           | 78South Africa                       | South Africa           | -30.5595 |   22.9375  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        1 |        1 |        1 |        3 |        3 |         7 |        13 |
| 187 | Confirmed |       0           | 79Bhutan                             | Bhutan                 |  27.5142 |   90.4336  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |         1 |         1 |
| 188 | Confirmed |       0           | 80Cameroon                           | Cameroon               |   3.848  |   11.5021  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        2 |        2 |         2 |         2 |
| 189 | Confirmed |       0           | 81Colombia                           | Colombia               |   4.5709 |  -74.2973  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |         3 |         9 |
| 190 | Confirmed |       0           | 82Costa Rica                         | Costa Rica             |   9.7489 |  -83.7534  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        5 |        9 |         9 |        13 |
| 191 | Confirmed |       0           | 83Peru                               | Peru                   |  -9.19   |  -75.0152  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        6 |        7 |        11 |        11 |
| 192 | Confirmed |       0           | 84Serbia                             | Serbia                 |  44.0165 |   21.0059  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |         5 |        12 |
| 193 | Confirmed |       0           | 85Slovakia                           | Slovakia               |  48.669  |   19.699   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        3 |        3 |         7 |        10 |
| 194 | Confirmed |       0           | 86Togo                               | Togo                   |   8.6195 |    0.8248  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |         1 |         1 |
| 195 | Confirmed |       0           | 87French Guiana                      | French Guiana          |   3.9339 |  -53.1258  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        5 |        5 |        5 |         5 |         5 |
| 196 | Confirmed |       0           | 88Malta                              | Malta                  |  35.9375 |   14.3754  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        3 |        3 |        3 |         5 |         6 |
| 197 | Confirmed |       0           | 89Martinique                         | Martinique             |  14.6415 |  -61.0242  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        2 |        2 |        2 |         2 |         3 |
| 198 | Confirmed |       0           | 90Bulgaria                           | Bulgaria               |  42.7339 |   25.4858  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        4 |        4 |         4 |         7 |
| 199 | Confirmed |       0           | 91Maldives                           | Maldives               |   3.2028 |   73.2207  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        4 |        4 |         6 |         8 |

In [24]:
df_ProvState = dfMain.dropna(how='any')

In [25]:
tab = tabulate(df_ProvState.head(n=200), tablefmt="pipe", headers="keys")
display.display(display.Markdown(tab))

|     | type      |   population size | Province/State                       | Country/Region         |      Lat |       Long |   1/22/20 |   1/23/20 |   1/24/20 |   1/25/20 |   1/26/20 |   1/27/20 |   1/28/20 |   1/29/20 |   1/30/20 |   1/31/20 |   2/1/20 |   2/2/20 |   2/3/20 |   2/4/20 |   2/5/20 |   2/6/20 |   2/7/20 |   2/8/20 |   2/9/20 |   2/10/20 |   2/11/20 |   2/12/20 |   2/13/20 |   2/14/20 |   2/15/20 |   2/16/20 |   2/17/20 |   2/18/20 |   2/19/20 |   2/20/20 |   2/21/20 |   2/22/20 |   2/23/20 |   2/24/20 |   2/25/20 |   2/26/20 |   2/27/20 |   2/28/20 |   2/29/20 |   3/1/20 |   3/2/20 |   3/3/20 |   3/4/20 |   3/5/20 |   3/6/20 |   3/7/20 |   3/8/20 |   3/9/20 |   3/10/20 |   3/11/20 |
|----:|:----------|------------------:|:-------------------------------------|:-----------------------|---------:|-----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|----------:|
|   0 | Confirmed |       0           | Chieti                               | Italia                 |  42.351  |   14.1675  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         9 |         9 |
|   1 | Confirmed |       0           | L'Aquila                             | Italia                 |  42.3512 |   13.3984  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         6 |
|   2 | Confirmed |       0           | Pescara                              | Italia                 |  42.4646 |   14.2136  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        18 |        18 |
|   3 | Confirmed |       0           | Teramo                               | Italia                 |  42.6589 |   13.7044  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         2 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         5 |         5 |
|   4 | Confirmed |       0           | In fase di definizione/aggiornamento | Italia                 |   0      |    0       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         5 |        21 |        13 |        42 |        53 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        42 |        40 |
|   5 | Confirmed |       0           | Matera                               | Italia                 |  40.6675 |   16.5979  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         3 |
|   6 | Confirmed |       0           | Potenza                              | Italia                 |  40.6395 |   15.8051  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         4 |         5 |
|   7 | Confirmed |       0           | Bolzano                              | Italia                 |  46.4993 |   11.3566  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        38 |        75 |
|   8 | Confirmed |       0           | Catanzaro                            | Italia                 |  38.906  |   16.5944  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         2 |
|   9 | Confirmed |       0           | Cosenza                              | Italia                 |  39.2931 |   16.2561  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         5 |
|  10 | Confirmed |       0           | Crotone                              | Italia                 |  39.0804 |   17.1254  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         1 |
|  11 | Confirmed |       0           | Reggio di Calabria                   | Italia                 |  38.1092 |   15.6435  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         6 |
|  12 | Confirmed |       0           | Vibo Valentia                        | Italia                 |  38.6762 |   16.1016  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         5 |
|  13 | Confirmed |       0           | Avellino                             | Italia                 |  40.914  |   14.7953  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         7 |
|  14 | Confirmed |       0           | Benevento                            | Italia                 |  41.1297 |   14.7815  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         4 |         2 |
|  15 | Confirmed |       0           | Caserta                              | Italia                 |  41.0747 |   14.3324  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        34 |        26 |
|  16 | Confirmed |       0           | Napoli                               | Italia                 |  40.8396 |   14.2508  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         4 |        13 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        61 |        96 |
|  17 | Confirmed |       0           | Salerno                              | Italia                 |  40.6782 |   14.7594  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        17 |        17 |
|  18 | Confirmed |       0           | Bologna                              | Italia                 |  44.4944 |   11.3417  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        86 |       108 |
|  19 | Confirmed |       0           | Ferrara                              | Italia                 |  44.836  |   11.6187  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         8 |        12 |
|  20 | Confirmed |       0           | Forlì-Cesena                         | Italia                 |  44.2227 |   12.0407  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        20 |        24 |
|  21 | Confirmed |       0           | Modena                               | Italia                 |  44.646  |   10.9262  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         8 |        18 |        18 |        22 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       127 |       163 |
|  22 | Confirmed |       0           | Parma                                | Italia                 |  44.8011 |   10.3283  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         4 |         8 |        10 |        27 |        35 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       325 |       378 |
|  23 | Confirmed |       0           | Piacenza                             | Italia                 |  45.0519 |    9.69263 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        18 |        28 |        63 |        89 |       138 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       633 |       664 |
|  24 | Confirmed |       0           | Ravenna                              | Italia                 |  44.4172 |   12.1991  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        24 |        31 |
|  25 | Confirmed |       0           | Reggio nell'Emilia                   | Italia                 |  44.6974 |   10.6301  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         4 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       104 |       114 |
|  26 | Confirmed |       0           | Rimini                               | Italia                 |  44.0609 |   12.5656  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         6 |         9 |        15 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       206 |       245 |
|  27 | Confirmed |       0           | Gorizia                              | Italia                 |  45.9415 |   13.6221  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        10 |        12 |
|  28 | Confirmed |       0           | Pordenone                            | Italia                 |  45.9544 |   12.66    |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         4 |        13 |
|  29 | Confirmed |       0           | Trieste                              | Italia                 |  45.6494 |   13.7681  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        53 |        57 |
|  30 | Confirmed |       0           | Udine                                | Italia                 |  46.0626 |   13.2348  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        26 |        44 |
|  31 | Confirmed |       0           | Frosinone                            | Italia                 |  41.6396 |   13.3512  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         8 |
|  32 | Confirmed |       0           | Latina                               | Italia                 |  41.4676 |   12.9037  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        11 |        11 |
|  33 | Confirmed |       0           | Rieti                                | Italia                 |  42.4049 |   12.8621  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         3 |
|  34 | Confirmed |       0           | Roma                                 | Italia                 |  41.8928 |   12.4837  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         3 |         3 |         3 |         6 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        76 |        99 |
|  35 | Confirmed |       0           | Viterbo                              | Italia                 |  42.4174 |   12.1047  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         5 |        10 |
|  36 | Confirmed |       0           | Genova                               | Italia                 |  44.4115 |    8.9327  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        42 |        63 |
|  37 | Confirmed |       0           | Imperia                              | Italia                 |  43.8857 |    8.02785 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        15 |        18 |
|  38 | Confirmed |       0           | La Spezia                            | Italia                 |  44.107  |    9.82819 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        15 |        19 |
|  39 | Confirmed |       0           | Savona                               | Italia                 |  44.3075 |    8.48111 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        10 |        18 |        18 |        41 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        38 |        42 |
|  40 | Confirmed |       0           | Bergamo                              | Italia                 |  45.6944 |    9.66842 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        18 |        20 |        72 |       103 |       110 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |      1472 |      1815 |
|  41 | Confirmed |       0           | Brescia                              | Italia                 |  45.5399 |   10.2191  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |        10 |        13 |        14 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       790 |      1351 |
|  42 | Confirmed |       0           | Como                                 | Italia                 |  45.81   |    9.08516 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        46 |        77 |
|  43 | Confirmed |       0           | Cremona                              | Italia                 |  45.1334 |   10.0242  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        53 |        57 |        91 |       123 |       136 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       957 |      1061 |
|  44 | Confirmed |       0           | Lecco                                | Italia                 |  45.8558 |    9.39339 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        89 |       113 |
|  45 | Confirmed |       0           | Lodi                                 | Italia                 |  45.3144 |    9.50372 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |       125 |       128 |       159 |       182 |       237 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       963 |      1035 |
|  46 | Confirmed |       0           | Mantova                              | Italia                 |  45.1573 |   10.7928  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       119 |       137 |
|  47 | Confirmed |       0           | Milano                               | Italia                 |  45.4668 |    9.19035 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         8 |         8 |        15 |        29 |        30 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       592 |       925 |
|  48 | Confirmed |       0           | Monza e della Brianza                | Italia                 |  45.5844 |    9.27358 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         4 |         5 |         6 |         6 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        65 |        85 |
|  49 | Confirmed |       0           | Pavia                                | Italia                 |  45.1851 |    9.16016 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        27 |        27 |        36 |        49 |        55 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       324 |       403 |
|  50 | Confirmed |       0           | Sondrio                              | Italia                 |  46.171  |    9.87147 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         3 |         3 |         3 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         7 |        13 |
|  51 | Confirmed |       0           | Varese                               | Italia                 |  45.817  |    8.82287 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         4 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        50 |        75 |
|  52 | Confirmed |       0           | Ancona                               | Italia                 |  43.6168 |   13.5189  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        81 |       110 |
|  53 | Confirmed |       0           | Ascoli Piceno                        | Italia                 |  42.8532 |   13.5769  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         1 |
|  54 | Confirmed |       0           | Fermo                                | Italia                 |  43.1606 |   13.7184  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         6 |         8 |
|  55 | Confirmed |       0           | Macerata                             | Italia                 |  43.3002 |   13.4531  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        11 |        17 |
|  56 | Confirmed |       0           | Pesaro e Urbino                      | Italia                 |  43.9101 |   12.9135  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         6 |        11 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       296 |       342 |
|  57 | Confirmed |       0           | Campobasso                           | Italia                 |  41.5577 |   14.6592  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        15 |        16 |
|  58 | Confirmed |       0           | Isernia                              | Italia                 |  41.588  |   14.2258  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  59 | Confirmed |       0           | Alessandria                          | Italia                 |  44.913  |    8.6154  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        65 |       105 |
|  60 | Confirmed |       0           | Asti                                 | Italia                 |  44.8991 |    8.20414 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        58 |        68 |
|  61 | Confirmed |       0           | Biella                               | Italia                 |  45.5665 |    8.05408 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        20 |        36 |
|  62 | Confirmed |       0           | Cuneo                                | Italia                 |  44.3933 |    7.55117 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        14 |        17 |
|  63 | Confirmed |       0           | Novara                               | Italia                 |  45.4459 |    8.62192 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        22 |        27 |
|  64 | Confirmed |       0           | Torino                               | Italia                 |  45.0733 |    7.68069 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         3 |         2 |        11 |        11 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       111 |       159 |
|  65 | Confirmed |       0           | Verbano-Cusio-Ossola                 | Italia                 |  45.9214 |    8.55108 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        11 |        13 |
|  66 | Confirmed |       0           | Vercelli                             | Italia                 |  45.324  |    8.42323 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        18 |        24 |
|  67 | Confirmed |       0           | Bari                                 | Italia                 |  41.1256 |   16.8674  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        15 |        18 |
|  68 | Confirmed |       0           | Barletta-Andria-Trani                | Italia                 |  41.2271 |   16.2952  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         4 |
|  69 | Confirmed |       0           | Brindisi                             | Italia                 |  40.6385 |   17.946   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         5 |        15 |
|  70 | Confirmed |       0           | Foggia                               | Italia                 |  41.4623 |   15.5431  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        23 |        24 |
|  71 | Confirmed |       0           | Lecce                                | Italia                 |  40.3535 |   18.1719  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        10 |        12 |
|  72 | Confirmed |       0           | Taranto                              | Italia                 |  40.4735 |   17.2324  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         3 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |         4 |
|  73 | Confirmed |       0           | Cagliari                             | Italia                 |  39.2153 |    9.11062 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        15 |        15 |
|  74 | Confirmed |       0           | Nuoro                                | Italia                 |  40.3232 |    9.3303  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         3 |        18 |
|  75 | Confirmed |       0           | Oristano                             | Italia                 |  39.9038 |    8.59118 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         2 |
|  76 | Confirmed |       0           | Sassari                              | Italia                 |  40.7267 |    8.55967 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         2 |
|  77 | Confirmed |       0           | Sud Sardegna                         | Italia                 |  39.1664 |    8.52624 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  78 | Confirmed |       0           | Agrigento                            | Italia                 |  37.3097 |   13.5846  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        11 |        16 |
|  79 | Confirmed |       0           | Caltanissetta                        | Italia                 |  37.4921 |   14.0618  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
|  80 | Confirmed |       0           | Catania                              | Italia                 |  37.5029 |   15.087   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        35 |        41 |
|  81 | Confirmed |       0           | Enna                                 | Italia                 |  37.5671 |   14.2791  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         1 |
|  82 | Confirmed |       0           | Messina                              | Italia                 |  38.194  |   15.5557  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         2 |         4 |
|  83 | Confirmed |       0           | Palermo                              | Italia                 |  38.1157 |   13.3624  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         3 |         3 |         3 |         3 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        10 |        15 |
|  84 | Confirmed |       0           | Ragusa                               | Italia                 |  36.9251 |   14.7307  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         1 |         1 |
|  85 | Confirmed |       0           | Siracusa                             | Italia                 |  37.0599 |   15.2933  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         2 |         3 |
|  86 | Confirmed |       0           | Trapani                              | Italia                 |  38.0185 |   12.5137  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         2 |
|  87 | Confirmed |       0           | Arezzo                               | Italia                 |  43.4664 |   11.8823  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        13 |        14 |
|  88 | Confirmed |       0           | Firenze                              | Italia                 |  43.7692 |   11.2559  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         2 |         1 |         7 |         5 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        61 |        71 |
|  89 | Confirmed |       0           | Grosseto                             | Italia                 |  42.7603 |   11.1136  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        10 |        12 |
|  90 | Confirmed |       0           | Livorno                              | Italia                 |  43.5523 |   10.3087  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        14 |        16 |
|  91 | Confirmed |       0           | Lucca                                | Italia                 |  43.8443 |   10.5015  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        37 |        43 |
|  92 | Confirmed |       0           | Massa Carrara                        | Italia                 |  44.0367 |   10.1417  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        37 |        40 |
|  93 | Confirmed |       0           | Pisa                                 | Italia                 |  43.7155 |   10.4013  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        31 |        34 |
|  94 | Confirmed |       0           | Pistoia                              | Italia                 |  43.9335 |   10.9173  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         0 |         1 |         1 |         1 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        21 |        32 |
|  95 | Confirmed |       0           | Prato                                | Italia                 |  43.8806 |   11.097   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         7 |        21 |
|  96 | Confirmed |       0           | Siena                                | Italia                 |  43.3182 |   11.3319  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        33 |        37 |
|  97 | Confirmed |       0           | Trento                               | Italia                 |  46.0689 |   11.1212  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        52 |        77 |
|  98 | Confirmed |       0           | Perugia                              | Italia                 |  43.1068 |   12.3882  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        13 |        26 |
|  99 | Confirmed |       0           | Terni                                | Italia                 |  42.5607 |   12.6467  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        24 |        18 |
| 100 | Confirmed |       0           | Aosta                                | Italia                 |  45.7375 |    7.32015 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        17 |        20 |
| 101 | Confirmed |       0           | Belluno                              | Italia                 |  46.1384 |   12.217   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        29 |        30 |
| 102 | Confirmed |       0           | Padova                               | Italia                 |  45.4069 |   11.8761  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        30 |        40 |        59 |        68 |        81 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       296 |       373 |
| 103 | Confirmed |       0           | Rovigo                               | Italia                 |  45.0711 |   11.7901  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        10 |        14 |
| 104 | Confirmed |       0           | Treviso                              | Italia                 |  45.6675 |   12.2451  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         2 |        22 |        23 |        35 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       158 |       185 |
| 105 | Confirmed |       0           | Venezia                              | Italia                 |  45.4349 |   12.3385  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         7 |         8 |        14 |        15 |        19 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |       152 |       179 |
| 106 | Confirmed |       0           | Verona                               | Italia                 |  45.4384 |   10.9935  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        96 |       110 |
| 107 | Confirmed |       0           | Vicenza                              | Italia                 |  45.5475 |   11.546   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         3 |         3 |         3 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        73 |        92 |
| 108 | Confirmed |       0           | 0Thailand                            | Thailand               |  15      |  101       |         2 |         3 |         5 |         7 |         8 |         8 |        14 |        14 |        14 |        19 |       19 |       19 |       19 |       25 |       25 |       25 |       25 |       32 |       32 |        32 |        33 |        33 |        33 |        33 |        33 |        34 |        35 |        35 |        35 |        35 |        35 |        35 |        35 |        35 |        37 |        40 |        40 |        41 |        42 |       42 |       43 |       43 |       43 |       47 |       48 |       50 |       50 |       50 |        53 |        59 |
| 109 | Confirmed |       0           | 1Japan                               | Japan                  |  36      |  138       |         2 |         1 |         2 |         2 |         4 |         4 |         7 |         7 |        11 |        15 |       20 |       20 |       20 |       22 |       22 |       45 |       25 |       25 |       26 |        26 |        26 |        28 |        28 |        29 |        43 |        59 |        66 |        74 |        84 |        94 |       105 |       122 |       147 |       159 |       170 |       189 |       214 |       228 |       241 |      256 |      274 |      293 |      331 |      360 |      420 |      461 |      502 |      511 |       581 |       639 |
| 110 | Confirmed |       0           | 2Singapore                           | Singapore              |   1.2833 |  103.833   |         0 |         1 |         3 |         3 |         4 |         5 |         7 |         7 |        10 |        13 |       16 |       18 |       18 |       24 |       28 |       28 |       30 |       33 |       40 |        45 |        47 |        50 |        58 |        67 |        72 |        75 |        77 |        81 |        84 |        84 |        85 |        85 |        89 |        89 |        91 |        93 |        93 |        93 |       102 |      106 |      108 |      110 |      110 |      117 |      130 |      138 |      150 |      150 |       160 |       178 |
| 111 | Confirmed |       0           | 3Nepal                               | Nepal                  |  28.1667 |   84.25    |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 112 | Confirmed |       0           | 4Malaysia                            | Malaysia               |   2.5    |  112.5     |         0 |         0 |         0 |         3 |         4 |         4 |         4 |         7 |         8 |         8 |        8 |        8 |        8 |       10 |       12 |       12 |       12 |       16 |       16 |        18 |        18 |        18 |        19 |        19 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        22 |        23 |        23 |        25 |       29 |       29 |       36 |       50 |       50 |       83 |       93 |       99 |      117 |       129 |       149 |
| 113 | Confirmed |       3.42642e+06 | British Columbia                     | Canada                 |  49.2827 | -123.121   |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        2 |        2 |        4 |        4 |        4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         5 |         5 |         5 |         5 |         6 |         6 |         6 |         6 |         7 |         7 |         7 |         7 |         8 |        8 |        8 |        9 |       12 |       13 |       21 |       21 |       27 |       32 |        32 |        39 |
| 114 | Confirmed |       7.02054e+06 | New South Wales                      | Australia              | -33.8688 |  151.209   |         0 |         0 |         0 |         0 |         3 |         4 |         4 |         4 |         4 |         4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |        4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |         4 |        6 |        6 |       13 |       22 |       22 |       26 |       28 |       38 |       48 |        55 |        65 |
| 116 | Confirmed |       3.32551e+06 | Queensland                           | Australia              | -28.0167 |  153.4     |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         2 |        3 |        2 |        2 |        3 |        3 |        4 |        5 |        5 |        5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         5 |         9 |        9 |        9 |       11 |       11 |       13 |       13 |       13 |       15 |       15 |        18 |        20 |
| 117 | Confirmed |       0           | 9Cambodia                            | Cambodia               |  11.55   |  104.917   |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        2 |        2 |         2 |         3 |
| 118 | Confirmed |       0           | 10Sri Lanka                          | Sri Lanka              |   7      |   81       |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         2 |
| 119 | Confirmed |       0           | 11Germany                            | Germany                |  51      |    9       |         0 |         0 |         0 |         0 |         0 |         1 |         4 |         4 |         4 |         5 |        8 |       10 |       12 |       12 |       12 |       12 |       13 |       13 |       14 |        14 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        16 |        17 |        27 |        46 |        48 |        79 |      130 |      159 |      196 |      262 |      482 |      670 |      799 |     1040 |     1176 |      1457 |      1908 |
| 120 | Confirmed |       0           | 12Finland                            | Finland                |  64      |   26       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         2 |         2 |         2 |         3 |        6 |        6 |        6 |        6 |       12 |       15 |       15 |       23 |       30 |        40 |        59 |
| 121 | Confirmed |       0           | 13United Arab Emirates               | United Arab Emirates   |  24      |   54       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         4 |         4 |         4 |        4 |        5 |        5 |        5 |        5 |        5 |        5 |        7 |        7 |         8 |         8 |         8 |         8 |         8 |         8 |         9 |         9 |         9 |         9 |         9 |         9 |        13 |        13 |        13 |        13 |        13 |        13 |        19 |        21 |       21 |       21 |       27 |       27 |       29 |       29 |       45 |       45 |       45 |        74 |        74 |
| 122 | Confirmed |       0           | 14Philippines                        | Philippines            |  13      |  122       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        2 |        2 |        2 |        2 |        2 |        3 |        3 |        3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |        3 |        3 |        3 |        3 |        3 |        5 |        6 |       10 |       20 |        33 |        49 |
| 123 | Confirmed |       0           | 15India                              | India                  |  21      |   78       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        2 |        3 |        3 |        3 |        3 |        3 |        3 |        3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |        3 |        5 |        5 |       28 |       30 |       31 |       34 |       39 |       43 |        56 |        62 |
| 124 | Confirmed |       0           | 16Italy                              | Italy                  |  43      |   12       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |        2 |        2 |        2 |        2 |        2 |        2 |        3 |        3 |        3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |         3 |        20 |        62 |       155 |       229 |       322 |       453 |       655 |       888 |      1128 |     1694 |     2036 |     2502 |     3089 |     3858 |     4636 |     5883 |     7375 |     9172 |     10149 |     12462 |
| 125 | Confirmed |       0           | 17Sweden                             | Sweden                 |  63      |   16       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         2 |         7 |         7 |        12 |       14 |       15 |       21 |       35 |       94 |      101 |      161 |      203 |      248 |       355 |       500 |
| 126 | Confirmed |       0           | 18Spain                              | Spain                  |  40      |   -4       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         6 |        13 |        15 |        32 |        45 |       84 |      120 |      165 |      222 |      259 |      400 |      500 |      673 |     1073 |      1695 |      2277 |
| 127 | Confirmed |       1.2978e+06  | South Australia                      | Australia              | -34.9285 |  138.601   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        1 |        2 |        2 |        2 |        2 |        2 |        2 |        2 |        2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         2 |         3 |        3 |        3 |        3 |        5 |        5 |        7 |        7 |        7 |        7 |         7 |         9 |
| 128 | Confirmed |       0           | 20Belgium                            | Belgium                |  50.8333 |    4       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        2 |        8 |       13 |       23 |       50 |      109 |      169 |      200 |      239 |       267 |       314 |
| 129 | Confirmed |       0           | 21Egypt                              | Egypt                  |  26      |   30       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |         1 |        2 |        2 |        2 |        2 |        3 |       15 |       15 |       49 |       55 |        59 |        60 |
| 130 | Confirmed |       0           | From Diamond Princess                | Australia              |  35.4437 |  139.638   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         4 |         7 |         7 |         7 |         7 |         7 |         8 |         8 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |
| 131 | Confirmed |       0           | 23Lebanon                            | Lebanon                |  33.8547 |   35.8623  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         2 |         2 |         2 |         4 |       10 |       13 |       13 |       13 |       16 |       22 |       22 |       32 |       32 |        41 |        61 |
| 132 | Confirmed |       0           | 24Iraq                               | Iraq                   |  33      |   44       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         5 |         7 |         7 |        13 |       19 |       26 |       32 |       35 |       35 |       40 |       54 |       60 |       60 |        71 |        71 |
| 133 | Confirmed |       0           | 25Oman                               | Oman                   |  21      |   57       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |         2 |         4 |         4 |         4 |         6 |        6 |        6 |       12 |       15 |       16 |       16 |       16 |       16 |       16 |        18 |        18 |
| 134 | Confirmed |       0           | 26Afghanistan                        | Afghanistan            |  33      |   65       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        4 |        4 |         5 |         7 |
| 135 | Confirmed |       0           | 27Bahrain                            | Bahrain                |  26.0275 |   50.55    |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        23 |        33 |        33 |        36 |        41 |       47 |       49 |       49 |       52 |       55 |       60 |       85 |       85 |       95 |       110 |       195 |
| 136 | Confirmed |       0           | 28Kuwait                             | Kuwait                 |  29.5    |   47.75    |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        11 |        26 |        43 |        45 |        45 |       45 |       56 |       56 |       56 |       58 |       58 |       61 |       64 |       64 |        69 |        72 |
| 137 | Confirmed |       0           | 29Algeria                            | Algeria                |  28.0339 |    1.6596  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |        1 |        3 |        5 |       12 |       12 |       17 |       17 |       19 |       20 |        20 |        20 |
| 138 | Confirmed |       0           | 30Croatia                            | Croatia                |  45.1    |   15.2     |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         3 |         5 |         6 |        7 |        7 |        9 |       10 |       10 |       11 |       12 |       12 |       12 |        14 |        19 |
| 139 | Confirmed |       0           | 31Switzerland                        | Switzerland            |  46.8182 |    8.2275  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         8 |         8 |        18 |       27 |       42 |       56 |       90 |      114 |      214 |      268 |      337 |      374 |       491 |       652 |
| 140 | Confirmed |       0           | 32Austria                            | Austria                |  47.5162 |   14.5501  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |         2 |         3 |         3 |         9 |       14 |       18 |       21 |       29 |       41 |       55 |       79 |      104 |      131 |       182 |       246 |
| 141 | Confirmed |       0           | 33Israel                             | Israel                 |  31      |   35       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |         1 |         2 |         3 |         4 |         7 |       10 |       10 |       12 |       15 |       20 |       37 |       43 |       61 |       61 |        83 |       109 |
| 142 | Confirmed |       0           | 34Pakistan                           | Pakistan               |  30.3753 |   69.3451  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |         2 |         2 |         4 |        4 |        4 |        5 |        5 |        5 |        6 |        6 |        6 |        6 |        16 |        19 |
| 143 | Confirmed |       0           | 35Brazil                             | Brazil                 | -14.235  |  -51.9253  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         2 |        2 |        2 |        2 |        4 |        4 |       13 |       13 |       20 |       25 |        31 |        38 |
| 144 | Confirmed |       0           | 36Georgia                            | Georgia                |  42.3154 |   43.3569  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |        3 |        3 |        3 |        3 |        4 |        4 |        4 |       13 |       15 |        15 |        24 |
| 145 | Confirmed |       0           | 37Greece                             | Greece                 |  39.0742 |   21.8243  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         3 |         4 |         4 |        7 |        7 |        7 |        9 |       31 |       45 |       46 |       73 |       73 |        89 |        99 |
| 146 | Confirmed |       0           | 38North Macedonia                    | North Macedonia        |  41.6086 |   21.7453  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        3 |        3 |        3 |        3 |         7 |         7 |
| 147 | Confirmed |       0           | 39Norway                             | Norway                 |  60.472  |    8.4689  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         6 |        15 |       19 |       25 |       32 |       56 |       87 |      108 |      147 |      176 |      205 |       400 |       598 |
| 148 | Confirmed |       0           | 40Romania                            | Romania                |  45.9432 |   24.9668  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         3 |         3 |        3 |        3 |        3 |        4 |        6 |        9 |        9 |       15 |       15 |        25 |        45 |
| 149 | Confirmed |       0           | 41Estonia                            | Estonia                |  58.5953 |   25.0136  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |        1 |        1 |        2 |        2 |        3 |       10 |       10 |       10 |       10 |        12 |        16 |
| 150 | Confirmed |       0           | 42Netherlands                        | Netherlands            |  52.1326 |    5.2913  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         6 |       10 |       18 |       24 |       38 |       82 |      128 |      188 |      265 |      321 |       382 |       503 |
| 151 | Confirmed |       0           | 43San Marino                         | San Marino             |  43.9424 |   12.4578  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |         1 |        1 |        8 |       10 |       16 |       21 |       21 |       23 |       36 |       36 |        51 |        62 |
| 152 | Confirmed |       0           | 44Belarus                            | Belarus                |  53.7098 |   27.9534  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        1 |        1 |        6 |        6 |        6 |        6 |        6 |        6 |         9 |         9 |
| 153 | Confirmed |       0           | 45Iceland                            | Iceland                |  64.9631 |  -19.0208  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        3 |        6 |       11 |       26 |       34 |       43 |       50 |       50 |       58 |        69 |        85 |
| 154 | Confirmed |       0           | 46Lithuania                          | Lithuania              |  55.1694 |   23.8813  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         3 |
| 155 | Confirmed |       0           | 47Mexico                             | Mexico                 |  23.6345 | -102.553   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         4 |        5 |        5 |        5 |        5 |        5 |        6 |        6 |        7 |        7 |         7 |         8 |
| 156 | Confirmed |       0           | 48New Zealand                        | New Zealand            | -40.9006 |  174.886   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        1 |        1 |        3 |        3 |        4 |        5 |        5 |        5 |         5 |         5 |
| 157 | Confirmed |       0           | 49Nigeria                            | Nigeria                |   9.082  |    8.6753  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        2 |         2 |         2 |
| 158 | Confirmed |       1.88282e+06 | Western Australia                    | Australia              | -31.9505 |  115.861   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         2 |        2 |        2 |        2 |        2 |        3 |        3 |        3 |        3 |        4 |         6 |         9 |
| 159 | Confirmed |       0           | 51Ireland                            | Ireland                |  53.1424 |   -7.6921  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        1 |        1 |        2 |        6 |        6 |       18 |       18 |       19 |       21 |        34 |        43 |
| 160 | Confirmed |       0           | 52Luxembourg                         | Luxembourg             |  49.8153 |    6.1296  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        1 |        1 |        1 |        1 |        1 |        2 |        2 |        3 |        3 |         5 |         7 |
| 161 | Confirmed |       0           | 53Monaco                             | Monaco                 |  43.7333 |    7.4167  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 162 | Confirmed |       0           | 54Qatar                              | Qatar                  |  25.3548 |   51.1839  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         1 |        3 |        3 |        7 |        8 |        8 |        8 |        8 |       15 |       18 |        24 |       262 |
| 163 | Confirmed |       0           | 55Ecuador                            | Ecuador                |  -1.8312 |  -78.1834  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        6 |        6 |        7 |       10 |       13 |       13 |       13 |       14 |       15 |        15 |        17 |
| 164 | Confirmed |       0           | 56Azerbaijan                         | Azerbaijan             |  40.1431 |   47.5769  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        3 |        3 |        3 |        3 |        6 |        6 |        9 |        9 |        9 |        11 |        11 |
| 165 | Confirmed |       0           | 57Armenia                            | Armenia                |  40.0691 |   45.0382  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 166 | Confirmed |       0           | 58Dominican Republic                 | Dominican Republic     |  18.7357 |  -70.1627  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        1 |        1 |        1 |        1 |        1 |        2 |        2 |        5 |        5 |         5 |         5 |
| 167 | Confirmed |       0           | 59Indonesia                          | Indonesia              |  -0.7893 |  113.921   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        2 |        2 |        2 |        2 |        4 |        4 |        6 |       19 |        27 |        34 |
| 168 | Confirmed |       0           | 60Portugal                           | Portugal               |  39.3999 |   -8.2245  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        2 |        2 |        5 |        8 |       13 |       20 |       30 |       30 |        41 |        59 |
| 169 | Confirmed |       0           | 61Andorra                            | Andorra                |  42.5063 |    1.5218  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 170 | Confirmed |  217344           | Tasmania                             | Australia              | -41.4545 |  145.971   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |        2 |        2 |         2 |         3 |
| 171 | Confirmed |       0           | 63Latvia                             | Latvia                 |  56.8796 |   24.6032  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |        2 |        6 |         8 |        10 |
| 172 | Confirmed |       0           | 64Morocco                            | Morocco                |  31.7917 |   -7.0926  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        1 |        1 |        2 |        2 |        2 |        2 |        2 |         3 |         5 |
| 173 | Confirmed |       0           | 65Saudi Arabia                       | Saudi Arabia           |  24      |   45       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        1 |        1 |        5 |        5 |        5 |       11 |       15 |        20 |        21 |
| 174 | Confirmed |       0           | 66Senegal                            | Senegal                |  14.4974 |  -14.4524  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        1 |        2 |        4 |        4 |        4 |        4 |        4 |        4 |         4 |         4 |
| 175 | Confirmed |       0           | 67Argentina                          | Argentina              | -38.4161 |  -63.6167  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        1 |        1 |        1 |        2 |        8 |       12 |       12 |        17 |        19 |
| 176 | Confirmed |       0           | 68Chile                              | Chile                  | -35.6751 |  -71.543   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        1 |        1 |        4 |        4 |        4 |        8 |        8 |        13 |        23 |
| 177 | Confirmed |       0           | 69Jordan                             | Jordan                 |  31.24   |   36.51    |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 178 | Confirmed |       0           | 70Ukraine                            | Ukraine                |  48.3794 |   31.1656  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 179 | Confirmed |       0           | 71Hungary                            | Hungary                |  47.1625 |   19.5033  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        2 |        2 |        2 |        4 |        7 |        9 |         9 |        13 |
| 180 | Confirmed |  137788           | Northern Territory                   | Australia              | -12.4634 |  130.846   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        1 |        1 |        0 |        0 |        0 |        0 |         1 |         1 |
| 181 | Confirmed |       0           | 73Liechtenstein                      | Liechtenstein          |  47.14   |    9.55    |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |        1 |        1 |         1 |         1 |
| 182 | Confirmed |       0           | 74Poland                             | Poland                 |  51.9194 |   19.1451  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        1 |        1 |        5 |        5 |       11 |       16 |        22 |        31 |
| 183 | Confirmed |       0           | 75Tunisia                            | Tunisia                |  34      |    9       |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |        2 |        2 |         5 |         7 |
| 184 | Confirmed |       0           | 76Bosnia and Herzegovina             | Bosnia and Herzegovina |  43.9159 |   17.6791  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        2 |        2 |        3 |        3 |        3 |         5 |         7 |
| 185 | Confirmed |       0           | 77Slovenia                           | Slovenia               |  46.1512 |   14.9955  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        2 |        7 |        7 |       16 |       16 |        31 |        57 |
| 186 | Confirmed |       0           | 78South Africa                       | South Africa           | -30.5595 |   22.9375  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        1 |        1 |        1 |        3 |        3 |         7 |        13 |
| 187 | Confirmed |       0           | 79Bhutan                             | Bhutan                 |  27.5142 |   90.4336  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |         1 |         1 |
| 188 | Confirmed |       0           | 80Cameroon                           | Cameroon               |   3.848  |   11.5021  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        2 |        2 |         2 |         2 |
| 189 | Confirmed |       0           | 81Colombia                           | Colombia               |   4.5709 |  -74.2973  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |         3 |         9 |
| 190 | Confirmed |       0           | 82Costa Rica                         | Costa Rica             |   9.7489 |  -83.7534  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        5 |        9 |         9 |        13 |
| 191 | Confirmed |       0           | 83Peru                               | Peru                   |  -9.19   |  -75.0152  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        6 |        7 |        11 |        11 |
| 192 | Confirmed |       0           | 84Serbia                             | Serbia                 |  44.0165 |   21.0059  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |         5 |        12 |
| 193 | Confirmed |       0           | 85Slovakia                           | Slovakia               |  48.669  |   19.699   |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        3 |        3 |         7 |        10 |
| 194 | Confirmed |       0           | 86Togo                               | Togo                   |   8.6195 |    0.8248  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        1 |        1 |        1 |        1 |         1 |         1 |
| 195 | Confirmed |       0           | 87French Guiana                      | French Guiana          |   3.9339 |  -53.1258  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        5 |        5 |        5 |         5 |         5 |
| 196 | Confirmed |       0           | 88Malta                              | Malta                  |  35.9375 |   14.3754  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        3 |        3 |        3 |         5 |         6 |
| 197 | Confirmed |       0           | 89Martinique                         | Martinique             |  14.6415 |  -61.0242  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        2 |        2 |        2 |         2 |         3 |
| 198 | Confirmed |       0           | 90Bulgaria                           | Bulgaria               |  42.7339 |   25.4858  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        4 |        4 |         4 |         7 |
| 199 | Confirmed |       0           | 91Maldives                           | Maldives               |   3.2028 |   73.2207  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        4 |        4 |         6 |         8 |
| 200 | Confirmed |       0           | 92Bangladesh                         | Bangladesh             |  23.685  |   90.3563  |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |         0 |        0 |        0 |        0 |        0 |        0 |        0 |        0 |        3 |        3 |         3 |         3 |

In [27]:
df_ProvState.to_csv('GlobalCOVID-DF.txt', sep='\t')

In [ ]:
col1 = df_ProvState.pop('type')
col2 = df_ProvState.pop('population size')
df_ProvState.insert(0, 'type', col1)
df_ProvState.insert(1, 'population size', col2)
